<a href="https://colab.research.google.com/github/mkandan/langchain_rag_tutorial/blob/main/langchain_rag_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install langchain langchain_community langchain_chroma langchain_openai langchainhub

In [7]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [10]:
# print(os.environ["LANGCHAIN_API_KEY"])

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4")

··········


In [9]:
# 1. use a web scraper/reader to read a url. bs_kwargs filters HTML content
# 2. loads web content into docs variable
# 3. inits a splitter that splits text into 1000 char chunks with 200 char overlap
# 4. run splitter on docs
# 5. use OpenAI embeddings on docs's chunks to create a ChromaDB vectore store
# 6. convert vectorstore to retrieval object
# 7. pull a prompt from langchain's hub
# 8. func that formats docs by concatenating with 2 newlines

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1, Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# 2
docs = loader.load()

# 3
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 4
splits = text_splitter.split_documents(docs)
# 5
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 6 Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
# 7
prompt = hub.pull("rlm/rag-prompt")

# 8
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a process used to break down complex tasks into smaller, simpler steps. This technique, often used in models to enhance performance, transforms a large task into multiple manageable tasks. Methods such as Chain of Thought (CoT) and Tree of Thoughts use this approach of task decomposition to create a step-by-step or tree structure of thoughts, respectively.'